In [1]:
import sympy as sp
import numpy as np
from sympy.physics.vector import dynamicsymbols
from sympy.physics.vector import time_derivative
from sympy.physics.vector import ReferenceFrame
N = ReferenceFrame('N')
import pylab as pl
import control
from sympy.physics.mechanics import *
from numpy.linalg import matrix_rank, eig
import WIP_utils as utils


In [2]:
RAD2DEG = 180/np.pi




def theta_print():
    print(theta[0])

def theta_q():
    global theta
    theta = [1,2,3,4]

In [3]:
theta_q()


In [4]:
theta_print()

1


In [5]:
time, traj = utils.Trapezoidal_Traj_Gen_Given_Amax_and_T(1.5,2,0.01)

In [6]:
traj

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.50000000e+00],
       [ 7.50000000e-05,  1.50000000e-02,  1.50000000e+00],
       [ 3.00000000e-04,  3.00000000e-02,  1.50000000e+00],
       [ 6.75000000e-04,  4.50000000e-02,  1.50000000e+00],
       [ 1.20000000e-03,  6.00000000e-02,  1.50000000e+00],
       [ 1.87500000e-03,  7.50000000e-02,  1.50000000e+00],
       [ 2.70000000e-03,  9.00000000e-02,  1.50000000e+00],
       [ 3.67500000e-03,  1.05000000e-01,  1.50000000e+00],
       [ 4.80000000e-03,  1.20000000e-01,  1.50000000e+00],
       [ 6.07500000e-03,  1.35000000e-01,  1.50000000e+00],
       [ 7.50000000e-03,  1.50000000e-01,  1.50000000e+00],
       [ 9.07500000e-03,  1.65000000e-01,  1.50000000e+00],
       [ 1.08000000e-02,  1.80000000e-01,  1.50000000e+00],
       [ 1.26750000e-02,  1.95000000e-01,  1.50000000e+00],
       [ 1.47000000e-02,  2.10000000e-01,  1.50000000e+00],
       [ 1.68750000e-02,  2.25000000e-01

In [7]:
traj_th1 = utils.Path_Gen(0, 1.5, traj[:,0])

In [11]:
traj_th1

array([0.00000000e+00, 0.00000000e+00, 1.12500000e-04, 4.50000000e-04,
       1.01250000e-03, 1.80000000e-03, 2.81250000e-03, 4.05000000e-03,
       5.51250000e-03, 7.20000000e-03, 9.11250000e-03, 1.12500000e-02,
       1.36125000e-02, 1.62000000e-02, 1.90125000e-02, 2.20500000e-02,
       2.53125000e-02, 2.88000000e-02, 3.25125000e-02, 3.64500000e-02,
       4.06125000e-02, 4.50000000e-02, 4.96125000e-02, 5.44500000e-02,
       5.95125000e-02, 6.48000000e-02, 7.03125000e-02, 7.60500000e-02,
       8.20125000e-02, 8.82000000e-02, 9.46125000e-02, 1.01250000e-01,
       1.08112500e-01, 1.15200000e-01, 1.22512500e-01, 1.30050000e-01,
       1.37812500e-01, 1.45800000e-01, 1.54012500e-01, 1.62450000e-01,
       1.71112500e-01, 1.80000000e-01, 1.89112500e-01, 1.98450000e-01,
       2.07951720e-01, 2.17461339e-01, 2.26970958e-01, 2.36480577e-01,
       2.45990196e-01, 2.55499815e-01, 2.65009434e-01, 2.74519053e-01,
       2.84028672e-01, 2.93538291e-01, 3.03047910e-01, 3.12557529e-01,
      